# Xarray-spatial
### User Guide: Zonal
-----

Xarray-spatial's zonal functions provide an easy way to generate statistics for zones within a raster aggregate. It's set up with a default set of calculations, or you can input any set of custom calculations you'd like to perform.

[Generate terrain](#Generate-Terrain-Data)
[Zonal Statistics](#Zonal-Statistics)

-----------


#### Let's use datashader to render our images...

To get started we'll bring in numpy and some functions from datashader to easily render our images.

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

import datashader as ds
from datashader.transfer_functions import shade
from datashader.transfer_functions import stack
from datashader.transfer_functions import dynspread
from datashader.transfer_functions import set_background
from datashader.colors import Elevation

## Generate Terrain Data

The rest of the geo-related functions focus on raster data or data that's been aggregates into the row-column grid of cells for an image raster. To demonstrate using these raster-based functions, we'll first use xarray-spatial's generate_terrain to generate a fake elevation terrain raster. We use datashader's Canvas as a quick base to set up a new raster.

In [ ]:
from xrspatial import generate_terrain

W = 800
H = 600

template_terrain = xr.DataArray(np.zeros((H, W)))
x_range=(-20, 20)
y_range=(-20, 20)

terrain = generate_terrain(template_terrain, x_range=x_range, y_range=y_range)

shade(terrain, cmap=['black', 'white'], how='linear')

We can also apply datashader's Elevation colormap imported above to give a more intuitive terrain image.

In [ ]:
shade(terrain, cmap=Elevation, how='linear')

## Zonal Statistics

Zonal statistics calculates summary statistics for specific areas or *zones* within an xarray.DataArray aggregate. Specific zones within an aggregate are defined by creating a corresponding aggregate of the same shape and setting the value at each cell to a unique non-zero integer representing a unique zone id. 

For example, if we set all the values in the top row of the zones aggregate to 3 and apply this to the original values aggregate, zonal stats will calculate statisitics for all the values in the corresponding top row of the values aggregate and return the results as stats for zone \#3. 

The output of zonal stats is in the form of a pandas DataFrame, with a row for each zone.

Let's set up an example.

Imagine you go on a six-day hike.
- We can represent the area with a terrain raster.
- In that terrain, we can represent each day's path as a line segment from your start to finish point.
- We can set this up with a pandas dataframe containing the start and finish points, which we then aggregate with Canvas.line. 

Let's take a look at these line zones overlayed on the fake terrain.

In [ ]:
from xrspatial import hillshade
from datashader.colors import Set1

terrain_shaded = shade(terrain, cmap=Elevation, alpha=128, how='linear')

illuminated = hillshade(terrain)
illuminated_shaded = shade(illuminated, cmap=['gray', 'white'], alpha=255, how='linear')

zone_df = pd.DataFrame({
   'x': [-11, -5, 4, 12, 14, 18, 19],
   'y': [-5, 4, 10, 13, 13, 13, 10],
   'trail_segement_id': [11, 12, 13, 14, 15, 16, 17]
})

cvs = ds.Canvas(plot_width=W, plot_height=H, x_range=x_range, y_range=y_range)

zones_agg = cvs.line(zone_df, 'x', 'y', ds.sum('trail_segement_id'))
zones_shaded = dynspread(shade(zones_agg, cmap=Set1), max_px=5, threshold=1)

stack(illuminated_shaded, terrain_shaded, zones_shaded)

Now, we can apply zonal stats, after quickly correcting for nan values.

In [ ]:
from xrspatial import zonal_stats

zones_agg.values = np.nan_to_num(zones_agg.values, copy=False).astype(int)
zonal_stats(zones_agg, terrain)

#### Calculate custom stats for each zone:

We can also put in our own set of stats calculations to perform instead of the default ones above. 
- We set up a dict with our desired functions and input that as the third argument to `zonal_stats`.
- Below, we try out a range function and min and max functions.

In [ ]:
custom_stats = dict(elevation_change=lambda zone: zone.max() - zone.min(),
                    elevation_min=np.min,
                    elevation_max=np.max)

zonal_stats(zones=zones_agg, values=terrain, stats_funcs=custom_stats)

Here the zones are defined by line segments, but they can be any spatial pattern or, more specifically, any region computable as a Datashader aggregate.

## Zonal crosstab

Zonal crosstab function can be used when we want to calculate cross-tabulated (categorical stats) areas between two datasets. As an example, assume we have gender data (male/female), and we want to see how all the gender groups distributed over all the zones defined above.

First, let's create a mock dataset for gender: 1 for male, and 2 for female. 

In [ ]:
# define a mask where 0s are water, and 1s are land areas
mask = terrain.data.astype(bool).astype(int)

# gender data where 0s are nodata, 1s are male, and 2s are female
# assume that the population only takes 10% overall, and the probability of male and female are equal.
genders = np.random.choice([0, 1, 2], p=[0.9, 0.05, 0.05], size=zones_agg.shape) * mask
genders_agg = xr.DataArray(genders, coords=terrain.coords, dims=terrain.dims)

# visualize the results
genders_shaded = shade(genders_agg, cmap=['white', 'blue', 'red'])
stack(genders_shaded, terrain_shaded)

Now, let's calculate the stats between the 2 datasets of genders and zones.

In [ ]:
from xrspatial import zonal_crosstab

zonal_crosstab(zones=zones_agg, values=genders_agg, zone_ids=[11, 12, 13, 14, 15, 16], cat_ids=[1, 2])